<a href="https://colab.research.google.com/github/BARATZL/MBTAheadway/blob/main/NCAAMB_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting the Outcome of March Madness Basketball Games

I love watching college basketball, particularly in March. My alma mater has never been in the "big dance", but the tournament has nevertheless always been very entertaining to me.

However, I do not know much about the sport itself. When it comes time to join in the office bracket pool, my bracket's success largely hinges on my luck, or the rushed google searches made just before filling out my predictions.

The above is the main motivation behind this project. Using Machine Learning concepts I have taken in during the fall semester, can I improve upon my bracket predictions from last year (where I more or less guessed)?

## Defining success




A simple way to define success of my model is to perform better than my predictions last year. I correctly guessed 42 out of 64 of the games, about 65% in total.

That sounds pretty decent for guessing, but my predicitions got worse after round 1 and 2 of the bracket. Equally weighting my predictions by each round, my accuracy looks something like this:



---



$\frac{1}{6}(R1 acc.+ R2 acc.+ R3acc...)$

Or,

$\frac{1}{6} (\frac{24}{32} + \frac{11}{16} + \frac{3}{8} + \frac{2}{4} + \frac{2}{2} + \frac{0}{1})$ = ~.55



---



So the standards I will initially aim for is a total accuracy higher than 65%, with an average accuracy across rounds higher than 55%.

# Data Sourcing and Formatting

There are two methods I have thought of that can be appropriately formatted for a model. The tabular data should be organized as follows:

Game | Team 1 | Team 1 Season Stat 1 | ... | Team 2 | Team 2 Season Stat 1 | ... | Team 1 Score | Team 2 Score | Team 1 Win (0 for no, 1 for yes)
----|----|----|----|----|----|----|----|----|---|
Purdue v. UConn | Purdue | x | ... | UConn | y | ... | 60 | 75 | 0
UConn v. Alabama | UConn | y | ... | Alabama | z | ... | 86 | 72 | 1
...|...|...|...|...|...|...|...|...|...

With this format, our model can either:

**1**. Predict the values that Team 1 and 2 score, with a subsequent function that confirms the outcome prediction the model is making.

**2**. Predict whether or not Team 1 wins.

We can assess both, but first we need to assemble our data. First, we need to create a table of season statistics.

Assembling this data across all years will be difficult, but should be possible through extracting data from [Sports Reference](https://www.sports-reference.com/cbb/). The below code begins this process.

In [185]:
from bs4 import BeautifulSoup, Comment
import numpy as np
import requests
import pandas as pd
url = 'https://www.sports-reference.com/cbb/seasons/men/2024-school-stats.html'
test = requests.get(url)

html_content = test.text
soup = BeautifulSoup(html_content, 'html.parser')
pretty_html = soup.prettify()
tables = soup.find_all('table')  # finding table in
headers = soup.find_all('th')

In [186]:
columnheaders = []
for i in headers:
 columnheaders.append(i.text)
columnheaders = columnheaders[13:50]
ncaa2324 = pd.DataFrame(columns = columnheaders)
ncaa2324.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   School  0 non-null      object
 1   G       0 non-null      object
 2   W       0 non-null      object
 3   L       0 non-null      object
 4   W-L%    0 non-null      object
 5   SRS     0 non-null      object
 6   SOS     0 non-null      object
 7           0 non-null      object
 8   W       0 non-null      object
 9   L       0 non-null      object
 10          0 non-null      object
 11  W       0 non-null      object
 12  L       0 non-null      object
 13          0 non-null      object
 14  W       0 non-null      object
 15  L       0 non-null      object
 16          0 non-null      object
 17  Tm.     0 non-null      object
 18  Opp.    0 non-null      object
 19          0 non-null      object
 20  MP      0 non-null      object
 21  FG      0 non-null      object
 22  FGA     0 non-null      object
 23  FG

In [187]:
rows = soup.find_all('tr')
for row in rows:
  cells = row.find_all('td')
  if cells == []:
    continue
  columndata = [col.text.strip() for col in cells]
  if "NCAA" in columndata[0]:
    ncaa2324.loc[len(ncaa2324)]=columndata

In [188]:
ncaa2324.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, 0 to 67
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   School  68 non-null     object
 1   G       68 non-null     object
 2   W       68 non-null     object
 3   L       68 non-null     object
 4   W-L%    68 non-null     object
 5   SRS     68 non-null     object
 6   SOS     68 non-null     object
 7           68 non-null     object
 8   W       68 non-null     object
 9   L       68 non-null     object
 10          68 non-null     object
 11  W       68 non-null     object
 12  L       68 non-null     object
 13          68 non-null     object
 14  W       68 non-null     object
 15  L       68 non-null     object
 16          68 non-null     object
 17  Tm.     68 non-null     object
 18  Opp.    68 non-null     object
 19          68 non-null     object
 20  MP      68 non-null     object
 21  FG      68 non-null     object
 22  FGA     68 non-null     object
 2

This table contains some of the statistics we would like to see for our team season data when we compile games from March Madness tournaments.

However, there's an issue with this webscraping method: the statistics listed include tournament games. This is problematic because we want the model to be useful prior to the tournament takes place.

If I train a model on data partially from tournaments, there's a chance that it will negatively affect the model when it is needed before teams even have a chance to compile tournament statistics.

One way to avoid this is to make sure that I only include rate/percentage statistics.

In [189]:
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace('NCAA$', '', regex=True)

In [190]:
ncaa2324.drop(columns=['W','L'], inplace=True)

In [191]:
columns_to_drop = ncaa2324.columns[5:9]
columns_to_drop
ncaa2324.drop(columns=columns_to_drop,inplace=True)

In [192]:
ncaa2324.drop(columns=['MP','FG','FGA','FT','FTA','3P','3PA'],inplace=True)

In [193]:
ncaa2324.iloc[:,1:] = ncaa2324.iloc[:,1:].astype(float)
column_ops = ['Tm.','Opp.','ORB','TRB','AST','STL','BLK','TOV','PF']
for col in column_ops:
  ncaa2324[col] = ncaa2324[col].astype(float)
  ncaa2324[col] = ncaa2324[col]/(ncaa2324['G'].astype(float))

ncaa2324.rename(columns={'Tm.':'PPG',
                         'Opp.':'PAPG',
                         'ORB':'ORBPG',
                         'TRB':'TRBPG',
                         'AST':'ASTPG',
                         'STL':'STLPG',
                         'BLK':'BLKPG',
                         'TOV':'TOVPG',
                         'PF':'PFPG'},inplace=True)

In [194]:
ncaa2324.drop(columns=['G','W-L%'],inplace=True)

Now I've eliminated most of the obvious indicators of postseason success that would not be available if we try to practically apply a model next year. Next, to create game data, and stitch these seasonal statistics onto the table with game data. I plan to pull game data again from Sports Reference.

In [195]:
mm2324 = pd.DataFrame(columns=['T1 Seed','T1 Name','T1 Score','T2 Seed','T2 Name','T2 Score'])

In [196]:
mm2324

,T1 Seed,T1 Name,T1 Score,T2 Seed,T2 Name,T2 Score


In [197]:
url2 = 'https://www.sports-reference.com/cbb/postseason/men/2024-ncaa.html'
test = requests.get(url2)

html_content = test.text
soup = BeautifulSoup(html_content, 'html.parser')
comments = soup.find_all(string=lambda text: isinstance(text, Comment))
for comment in comments:
    if "game" in comment:
        next_element = comment.find_next_sibling()
        try:
          seed1 = next_element.find('span').text.strip()
        except:
          break
        name1 = next_element.find('a', href=True).text.strip()
        try:
          score1 = next_element.find_all('a',href=True)[1].text.strip()
        except:
          continue
        third_element = next_element.find_next_sibling()
        seed2 = third_element.find('span').text.strip()
        name2 = third_element.find('a', href=True).text.strip()
        score2 = third_element.find_all('a',href=True)[1].text.strip()
        mm2324.loc[len(mm2324)] = [seed1,name1,score1,seed2,name2,score2]
mm2324.loc[len(mm2324)] = [1,'UConn',75,1,'Purdue',60]

In [199]:
mm2324['Game'] = mm2324['T1 Name'] + f' ('+mm2324['T1 Seed'].astype(str)+')' ' v. ' + mm2324['T2 Name'] + ' ('+mm2324['T2 Seed'].astype(str)+')'
mm2324.set_index('Game',inplace=True)
mm2324

,T1 Seed,T1 Name,T1 Score,T2 Seed,T2 Name,T2 Score
Game,,,,,,
UConn (1) v. Stetson (16),1,UConn,91,16,Stetson,52
Florida Atlantic (8) v. Northwestern (9),8,Florida Atlantic,65,9,Northwestern,77
San Diego State (5) v. UAB (12),5,San Diego State,69,12,UAB,65
Auburn (4) v. Yale (13),4,Auburn,76,13,Yale,78
BYU (6) v. Duquesne (11),6,BYU,67,11,Duquesne,71
...,...,...,...,...,...,...
Clemson (6) v. Arizona (2),6,Clemson,77,2,Arizona,72
Alabama (4) v. Clemson (6),4,Alabama,89,6,Clemson,82
UConn (1) v. Alabama (4),1,UConn,86,4,Alabama,72


Now that the game table has been created, the seasonal stats for the teams can be inserted into the table.

In [182]:
for index, row in mm2324.iterrows():
  t1 = row['T1 Name']
  t2 = row['T2 Name']
  if t1 in ncaa2324['School'].values and t2 in ncaa2324['School'].values:
    t1_stats = ncaa2324[ncaa2324['School']==t1].iloc[:,1:].values

,School,SRS,SOS,PPG,PAPG,FG%,3P%,FT%,ORBPG,TRBPG,ASTPG,STLPG,BLKPG,TOVPG,PFPG
0,Akron,2.77,-2.08,73.628571,66.171429,0.453,0.324,0.727,10.371429,36.514286,13.000000,5.628571,2.857143,11.257143,16.657143
1,Alabama,20.69,11.8,90.135135,81.243243,0.476,0.373,0.772,12.756757,39.648649,15.864865,6.918919,4.378378,11.837838,19.837838
2,Arizona,24.54,9.45,87.138889,72.055556,0.484,0.366,0.717,13.083333,42.583333,18.472222,8.333333,3.694444,11.944444,16.388889
3,Auburn,22.46,7.66,83.114286,68.314286,0.476,0.352,0.75,11.228571,37.800000,17.771429,7.371429,6.142857,10.685714,19.371429
4,Baylor,19.5,10.71,80.400000,71.114286,0.483,0.389,0.732,11.400000,35.114286,14.685714,6.742857,3.142857,12.028571,16.485714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Wagner,-10.64,-8.58,63.727273,63.121212,0.396,0.327,0.704,11.181818,35.848485,12.757576,5.909091,2.303030,9.878788,15.363636
64,Washington State,14.02,7.19,73.542857,66.714286,0.463,0.339,0.704,11.171429,37.628571,12.428571,5.171429,4.914286,10.942857,16.628571
65,Western Kentucky,1.19,-2.91,80.235294,74.441176,0.466,0.34,0.72,10.617647,39.941176,13.352941,7.647059,3.235294,13.823529,18.382353
66,Wisconsin,16.01,11.35,74.666667,70.000000,0.461,0.349,0.755,10.055556,34.000000,12.722222,6.222222,1.638889,9.944444,16.500000
